In [2]:
import requests
import time
import json
import numpy as np
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
import os

In [3]:
topic_name = 'volume_data'
bootstrap_servers = ['localhost:9092'] # адрес сервера Kafka
# csv_path = '~/3_8/file.csv'

In [4]:
# создаем директорию, если ее еще нет
if not os.path.exists('/Users/dja_tiger/3_8'):
    os.mkdir('/Users/dja_tiger/3_8')

csv_path = '/Users/dja_tiger/3_8/data.csv'

In [7]:
# Функция для агрегации данных об объемах торгов до окна в 4 часа и сохранения промежуточных результатов в csv
def aggregate_and_save_to_csv(data, csv_path):
    try:
        # Проверяем существование csv-файла
        if os.path.exists(csv_path):
            existing_data = pd.read_csv(csv_path, index_col=0)
        else:
            existing_data = pd.DataFrame()

        while True:
            volume_data = json.loads(data)

            for exchange_id, volume_one_day_data in volume_data.items():
                volume_one_day_data = np.array(volume_one_day_data, dtype=float)
                volume_four_hour_data = np.mean(np.reshape(volume_one_day_data, (-1, 6)), axis=1)

                volume_data_dict = {'exchange': exchange_id,
                                    'volume_1': volume_four_hour_data[0],
                                    'volume_2': volume_four_hour_data[1],
                                    'volume_3': volume_four_hour_data[2],
                                    'volume_4': volume_four_hour_data[3],
                                    'volume_5': volume_four_hour_data[4],
                                    'volume_6': volume_four_hour_data[5]}

                # Добавляем новые данные к уже существующим
                existing_data = pd.concat([existing_data, pd.DataFrame(volume_data_dict, index=[0])], ignore_index=True)
                print("Данные добавлены")
                print(existing_data.shape[0])

                # Если количество строк в файле кратно 5, сохраняем данные в csv
                if existing_data.shape[0] % 5 == 0:
                    existing_data.to_csv(csv_path, index=False)
                    print("Данные сохранены в CSV")

    # volume_data_dict.update({'exchange': exchange_id,
            #                          "volume_1": volume_four_hour_data[0],
            #                          "volume_2": volume_four_hour_data[1],
            #                          "volume_3": volume_four_hour_data[2],
            #                          "volume_4": volume_four_hour_data[3],
            #                          "volume_5": volume_four_hour_data[4],
            #                          "volume_6": volume_four_hour_data[5]})




            # volume_data_dict = {'exchange': exchange_id, 'volume_1': volume_four_hour_data[0], 'volume_2': volume_four_hour_data[1],
            #                     'volume_3': volume_four_hour_data[2], 'volume_4': volume_four_hour_data[3],
            #                     'volume_5': volume_four_hour_data[4], 'volume_6': volume_four_hour_data[5]}




            # if os.path.exists(csv_path):
            #     existing_data = pd.read_csv(csv_path, index_col=0)
            #     existing_data = existing_data.append(volume_data_dict, ignore_index=True)
            #     print("Данные добавлены")
            #     # Если количество строк в файле кратно 5, сохраняем данные в csv
            #     if existing_data.shape[0] % 5 == 0:
            #         existing_data.to_csv(csv_path, index=False)
            #         print("Данные сохранены в CSV")
            # else:
            #     existing_data = pd.DataFrame()
            #     existing_data = existing_data.append(volume_data_dict, ignore_index=True)





# except Exception as e:
#     print(f"An error occurred: {e}")

# finally:
#     if not existing_data.empty:
#         existing_data.to_csv(csv_path, index=False)
#         print(f"CSV file saved to {csv_path}")


    # Если файл уже существует, читаем его и добавляем новые данные
    #         if os.path.exists(csv_path):
    #             existing_data = pd.read_csv(csv_path, index_col=0)
    #             # existing_data = existing_data.append(volume_data_dict, ignore_index=True)
    #             existing_data = pd.concat([existing_data, pd.DataFrame(volume_data_dict, index=[0])], ignore_index=True)
    #
    #             print("Данные добавлены")
    #
    #             # Если количество строк в файле кратно 5, сохраняем данные в csv
    #             # if existing_data.shape[0] % 5 == 0:
    #             if len(existing_data) % 5 == 0:
    #                 existing_data.to_csv(csv_path, index=False)
    #                 print("Данные сохранены в CSV")
    #
    #         # Если файл не существует, создаем новый и записываем данные
    #         else:
    #             new_data = pd.DataFrame(volume_data_dict, index=[0])
    #             new_data.to_csv(csv_path, index=False)
    #             print("Создан файл CSV и записаны данные")


    except Exception as e:
        print(f"An error occurred: {e}")
    # finally:
    #     if os.path.exists(csv_path):
    #         print(f"CSV file saved to {csv_path}")


# def aggregate_and_save_to_csv(data, csv_path):
#
    # try:
    #     volume_data = json.loads(data)
    #     exchange_id = list(volume_data.keys())[0]
    #     volume_one_day_data = np.array(list(volume_data.values())[0], dtype=float)
    #     volume_four_hour_data = np.mean(np.reshape(volume_one_day_data, (-1, 6)), axis=1)
    #     volume_data_dict = {'exchange': exchange_id, 'volume_1': volume_four_hour_data[0], 'volume_2': volume_four_hour_data[1],
    #                         'volume_3': volume_four_hour_data[2], 'volume_4': volume_four_hour_data[3],
    #                         'volume_5': volume_four_hour_data[4], 'volume_6': volume_four_hour_data[5]}
    #
    #     # Если файл уже существует, читаем его и добавляем новые данные
    #     if os.path.exists(csv_path):
    #         existing_data = pd.read_csv(csv_path, index_col=0)
    #         existing_data = existing_data.append(volume_data_dict, ignore_index=True)
    #         print("Данные добавлены")
    #
    #         # Если количество строк в файле кратно 5, сохраняем данные в csv
    #         if existing_data.shape[0] % 5 == 0:
    #             existing_data.to_csv(csv_path, index=False)
    #             print("Данные сохранены в CSV")
    #
    #     # Если файл не существует, создаем новый и записываем данные
    #     else:
    #         new_data = pd.DataFrame(volume_data_dict, index=[0])
    #         new_data.to_csv(csv_path, index=False)
    #         print("Создан файл CSV и записаны данные")
    #
    # # except (TypeError, ValueError):
    # #     print(f"Failed to process data: {data}")
    # except Exception as e:
    #     print(f"An error occurred: {e}")
    # finally:
    #     if os.path.exists(csv_path):
    #         print(f"CSV file saved to {csv_path}")

In [8]:
consumer = KafkaConsumer(topic_name, bootstrap_servers=bootstrap_servers, auto_offset_reset='earliest',
                         enable_auto_commit=True, group_id='my-group', value_deserializer=lambda x: x.decode('utf-8'))

# for message in consumer:
#     volume_data = message.value
#     aggregate_and_save_to_csv(volume_data, csv_path)
#
# # Закрываем соединение с Kafka
# consumer.close()
#
try:
    for message in consumer:
        volume_data = message.value
        aggregate_and_save_to_csv(volume_data, csv_path)
except KeyboardInterrupt:
    print('User interrupted')
finally:
    # Закрываем соединение с Kafka
    consumer.close()

Данные добавлены
44
Данные добавлены
45
Данные сохранены в CSV
Данные добавлены
46
Данные добавлены
47
Данные добавлены
48
Данные добавлены
49
Данные добавлены
50
Данные сохранены в CSV
Данные добавлены
51
Данные добавлены
52
Данные добавлены
53
Данные добавлены
54
Данные добавлены
55
Данные сохранены в CSV
Данные добавлены
56
Данные добавлены
57
Данные добавлены
58
Данные добавлены
59
Данные добавлены
60
Данные сохранены в CSV
Данные добавлены
61
Данные добавлены
62
Данные добавлены
63
Данные добавлены
64
Данные добавлены
65
Данные сохранены в CSV
Данные добавлены
66
Данные добавлены
67
Данные добавлены
68
Данные добавлены
69
Данные добавлены
70
Данные сохранены в CSV
Данные добавлены
71
Данные добавлены
72
Данные добавлены
73
Данные добавлены
74
Данные добавлены
75
Данные сохранены в CSV
Данные добавлены
76
Данные добавлены
77
Данные добавлены
78
Данные добавлены
79
Данные добавлены
80
Данные сохранены в CSV
Данные добавлены
81
Данные добавлены
82
Данные добавлены
83
Данные добавлены